In [ ]:
def category_transition(cat_reIndex_mapping,count_dict,poi_cat_distrib,first_mall_index):    
    category_transition= np.zeros([len(cat_reIndex_mapping), len(cat_reIndex_mapping)])
    sum_dict=sum(count_dict.values())
    for (m,n) in count_dict.keys():
        #print(m)
        if (m >(first_mall_index-1)) & (n<first_mall_index):
            for cate_id in poi_cat_distrib[m].keys():
                total_num=sum(poi_cat_distrib[m].values())
                ratio=poi_cat_distrib[m][cate_id] / total_num 
                category_transition[cate_id][n]+=ratio*count_dict[m,n]
        else:
            if (m <first_mall_index)& (n>(first_mall_index-1)):
                for cate_id in poi_cat_distrib[n].keys():
                    total_num=sum(poi_cat_distrib[n].values())
                    ratio=poi_cat_distrib[n][cate_id] / total_num 
                    category_transition[m][cate_id]+=ratio*count_dict[m,n]
            else:
                if (m>(first_mall_index-1)) & (n>(first_mall_index-1)):
                    total_num1=sum(poi_cat_distrib[m].values())
                    total_num2=sum(poi_cat_distrib[n].values())
                    for cate_id1 in poi_cat_distrib[m].keys():
                        ratio1=poi_cat_distrib[m][cate_id1] / total_num1
                        for cate_id2 in poi_cat_distrib[n].keys():
                            ratio2=poi_cat_distrib[n][cate_id2]/ total_num2 
                            category_transition[cate_id1][cate_id2]+=ratio1*ratio2*count_dict[m,n]
                else:
                     category_transition[m][n]=count_dict[m,n]
    category_transition=category_transition/sum_dict
    return category_transition

In [ ]:
x_seq = tf.placeholder(tf.float32, shape = [None, None, seq_size]) 
x_time = tf.placeholder(tf.int32, shape = [None, None]) 
x_weekend = tf.placeholder(tf.int32, shape = [None, None]) 
x_type = tf.placeholder(tf.int32, shape = [None, None]) 
y_time = tf.placeholder(tf.int32, shape = [None, 1]) 
y_weekend = tf.placeholder(tf.int32, shape = [None, 1]) 
y_type = tf.placeholder(tf.int32, shape = [None, 1])
y_seq = tf.placeholder(tf.float32, shape = [None, 1, seq_size])

 

neg_seq = tf.placeholder(tf.float32, [None, neg_num, seq_size]) 
neg_time = tf.placeholder(tf.int32, [None, neg_num]) 
neg_weekend = tf.placeholder(tf.int32, [None, neg_num]) 
neg_type = tf.placeholder(tf.int32, [None, neg_num]) 



user_emb = tf.Variable(tf.random_uniform([user_size, hidden_size], -1.0, 1.0))
seq_emb_begin = tf.Variable(tf.random_uniform([seq_size, hidden_size], -1.0, 1.0))
seq_emb=tf.matmul(tf.cast(seq_transition,tf.float32),seq_emb_begin)+seq_emb_begin
type_emb = tf.Variable(tf.random_uniform([2, hidden_size], -1.0, 1.0))
time_emb = tf.Variable(tf.random_uniform([time_size, hidden_size], -1.0, 1.0))
weekend_emb = tf.Variable(tf.random_uniform([weekend_size, hidden_size], -1.0, 1.0))



init_weight = tf.truncated_normal([hidden_size, hidden_size], stddev = 1.0/np.sqrt(hidden_size))
W_time = tf.Variable(init_weight)
W_weekend = tf.Variable(init_weight)
W_seq = tf.Variable(init_weight)
W_type = tf.Variable(init_weight)
W_h_c = tf.Variable(init_weight)
W_h_t= tf.Variable(tf.truncated_normal([1, hidden_size], stddev = 1.0/np.sqrt(hidden_size)))

input_x_seq = tf.matmul(x_seq, tf.expand_dims(seq_emb, 0)) 
input_x_time = tf.nn.embedding_lookup(time_emb, x_time) 
input_x_weekend = tf.nn.embedding_lookup(weekend_emb, x_weekend)
input_x_type = tf.nn.embedding_lookup(type_emb, x_type) 

inputs_x=tf.matmul(input_x_time,tf.expand_dims(W_time,0)) +tf.matmul(input_x_weekend, tf.expand_dims(W_weekend,0))+ tf.matmul(input_x_seq, tf.expand_dims(W_seq,0)) + tf.matmul(input_x_type, tf.expand_dims(W_type,0)) 


input_y_seq = tf.matmul(y_seq, tf.expand_dims(seq_emb, 0)) 
input_y_time = tf.nn.embedding_lookup(time_emb, y_time) 
input_y_weekend = tf.nn.embedding_lookup(weekend_emb, y_weekend)
input_y_type = tf.nn.embedding_lookup(type_emb, y_type) 

inputs_y=tf.matmul(input_y_time, tf.expand_dims(W_time,0)) + tf.matmul(input_y_weekend, tf.expand_dims(W_weekend,0)) +tf.matmul(input_y_seq, tf.expand_dims(W_seq,0)) + tf.matmul(input_y_type, tf.expand_dims(W_type,0)) 


input_neg_seq = tf.matmul(neg_seq, tf.expand_dims(seq_emb, 0)) 
input_neg_time = tf.nn.embedding_lookup(time_emb, y_time) 
input_neg_weekend = tf.nn.embedding_lookup(weekend_emb, y_weekend) 
input_neg_type = tf.nn.embedding_lookup(type_emb, neg_type) 

inputs_neg = tf.matmul(input_neg_time, tf.expand_dims(W_time,0)) + tf.matmul(input_neg_weekend, tf.expand_dims(W_weekend,0))+ tf.matmul(input_neg_seq, tf.expand_dims(W_seq,0)) + tf.matmul(input_neg_type, tf.expand_dims(W_type,0))  


In [ ]:

with tf.variable_scope("rnn"):
    cell = tf.nn.rnn_cell.BasicLSTMCell(hidden_size, state_is_tuple=True)
    
    cell= tf.nn.rnn_cell.DropoutWrapper(cell, output_keep_prob=keep_prob)
    
    cell= tf.nn.rnn_cell.MultiRNNCell([cell] * RNN_stack_layers
                                      
    initial_state= cell.zero_state(batch_size, tf.float32)
                                      
    outputs, states = tf.nn.dynamic_rnn(cell, inputs = inputs_x, initial_state = initial_state)
                                      
                                      
                                      
regularization = tf.nn.l2_loss(W_time) + tf.nn.l2_loss(W_weekend) +tf.nn.l2_loss(W_seq) + tf.nn.l2_loss(seq_emb) + tf.nn.l2_loss(time_emb) +tf.nn.l2_loss(weekend_emb)+ tf.nn.l2_loss(W_h_c) + tf.nn.l2_loss(W_h_t) + tf.nn.l2_loss(W_type) + tf.nn.l2_loss(type_emb)   
                                     
                                      
final_output= tf.expand_dims(tf.transpose(outputs, [1, 0, 2])[-1], 1)  
                                                                         
output_h_c = tf.matmul(final_output, tf.expand_dims(W_h_c,0))
                                      
seq = tf.matmul(output_h_c, tf.transpose(inputs_y, [0, 2, 1])) 
                                      
seq_neg = tf.matmul(output_h_c, tf.transpose(inputs_neg, [0, 2, 1])) 
                                      
loss_seq = tf.reduce_sum(1 + tf.log(tf.exp(-(tf.tile(seq, [0, 0, neg_num]) - seq_neg)))) 

                                      

output_h_t = tf.matmul(final_output, tf.expand_dims(tf.transpose(W_h_t),0)) 
                                      
pred_t = tf.reduce_sum(tf.sigmoid(output_h_t)) 
                                      
loss_type = - (tf.cast(tf.reduce_sum(y_type), tf.float32) * tf.log(pred_t) + tf.cast((1-tf.reduce_sum(y_type)), tf.float32) * tf.log(1 - pred_t))

#loss

total_loss = w_c * loss_seq + w_t * loss_type + reg_beta * regularization

optimizer = tf.train.AdamOptimizer(learning_rate=lr)

gradients, variables = zip(*optimizer.compute_gradients(total_loss))

gradients, _ = tf.clip_by_global_norm(gradients, 5.0)

train = optimizer.apply_gradients(zip(gradients, variables))



In [ ]:
# training
saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    entropy_loss = []
    prev_loss = 10000
    try:    
        for i in range(iter_num):
            user_counter = 0

            sample_counter = 0

            iter_total_loss = 0.0

            for user_training_samples in all_training_samples:

                for sample in user_training_samples:
                    
                    feed_dict = data_feeder(sample)

                    _, _loss = sess.run([train, total_loss], 

                                        {x_time: feed_dict['x_time'], 
                                         x_weekend: feed_dict['x_weekend'],
                                         x_seq: feed_dict['x_seq'], 
                                         x_type: feed_dict['x_type'],                               
                                         y_time: feed_dict['y_time'], 
                                         y_weekend: feed_dict['y_weekend'], 
                                         y_seq: feed_dict['y_seq'], 
                                         y_type: feed_dict['y_type'],
                                         neg_seq: feed_dict['neg_seq'],
                                         neg_type: feed_dict['neg_type'] })
                    iter_total_loss += _loss

                    sample_counter += 1                  

                user_counter += 1

            avg_loss = iter_total_loss / sample_counter
            entropy_loss.append(avg_loss)
            delta=prev_loss - avg_loss
            if delta > break_threshold: 
                prev_loss = avg_loss  
            else: 
                raise StopIteration
    except StopIteration:
     